In [5]:
from pydantic import BaseModel, ValidationError, Field, EmailStr
from typing import List, Literal, Optional
from datetime import date
from dotenv import load_dotenv

import openai
import json
import os
import sys

try: 
    dotenv_loaded = load_dotenv()
    if not dotenv_loaded:
        print("Warning: no .env file found or loaded.")
except Exception as e:
    print(f"Error loading .env file: {e}")
    sys.exit(1)
    
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    print("Error: OPENAI_API_KEY not found in environment variables")
    print("Please create a .env file with: OPENAI_API_KEY=your_api_key_here")
    sys.exit(1)
    
try: 
    client = openai.OpenAI(api_key=api_key)
    print("OpenAI client successfully initialized.")
except Exception as e:
    print(f"Error initializing OpenAI client: {e}")
    sys.exit(1)

OpenAI client successfully initialized.


In [7]:
class UserInput(BaseModel):
    name: str
    email: str
    query: str
    order_id: Optional[int] = Field(
        None, 
        description="5 digit (not zero) Order ID if available",
        lower_bound = 10000,
        upper_bound = 99999,
    )
    purchase_date: Optional[date] = None


user_input_json = """
{
    "name": "John Doe",
    "email": "john.doe@example.com",
    "query": "I need assistance retrieving my order invoice",
    "order_id": null,
    "purchase_date": null
}
"""

user_input = UserInput.model_validate_json(user_input_json)

In [8]:
class UserQuery(UserInput):
    priority: str = Field(
        ..., description="Priority level: low, medium, high",
    )
    catetory: Literal[
        'refund_request',
        'information_request',
        'other'
    ]
    is_compliant: bool = Field(
        ..., description="Whether the query is compliant or not",
    )
    tags: List[str] = Field(
        description="List of tags associated with the query",
    )

In [11]:
model_schema = json.dumps(UserQuery.model_json_schema(), indent = 2)
print(f"model schema:\n{model_schema}")

model schema:
{
  "properties": {
    "name": {
      "title": "Name",
      "type": "string"
    },
    "email": {
      "title": "Email",
      "type": "string"
    },
    "query": {
      "title": "Query",
      "type": "string"
    },
    "order_id": {
      "anyOf": [
        {
          "type": "integer"
        },
        {
          "type": "null"
        }
      ],
      "default": null,
      "description": "5 digit (not zero) Order ID if available",
      "lower_bound": 10000,
      "title": "Order Id",
      "upper_bound": 99999
    },
    "purchase_date": {
      "anyOf": [
        {
          "format": "date",
          "type": "string"
        },
        {
          "type": "null"
        }
      ],
      "default": null,
      "title": "Purchase Date"
    },
    "priority": {
      "description": "Priority level: low, medium, high",
      "title": "Priority",
      "type": "string"
    },
    "catetory": {
      "enum": [
        "refund_request",
        "information_r

In [15]:
def call_llm(prompt: str, model: str = "gpt-4o") -> str:
    response = client.chat.completions.create(
        model = model,
        messages = [{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

In [16]:
def validate_with_model(data_model, llm_response):
    try:
        validated_data = data_model.model_validate_json(llm_response)
        print(f"Data validation successful.")
        print(validated_data.model_dump_json(indent = 2))
        return validated_data, None
    except Exception as e:
        print(f"Data validation failed: {e}")
        error_message = f"This response generated a validation error: {e}"
        return None, error_message